#Imports

In [ ]:
import pandas
import numpy
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

In [ ]:
car_data = pandas.read_csv('quikr_car.csv')
car_data.head()

,name,company,year,Price,kms_driven,fuel_type
0,Hyundai Santro Xing XO eRLX Euro III,Hyundai,2007,"80,000","45,000 kms",Petrol
1,Mahindra Jeep CL550 MDI,Mahindra,2006,"4,25,000",40 kms,Diesel
2,Maruti Suzuki Alto 800 Vxi,Maruti,2018,Ask For Price,"22,000 kms",Petrol
3,Hyundai Grand i10 Magna 1.2 Kappa VTVT,Hyundai,2014,"3,25,000","28,000 kms",Petrol
4,Ford EcoSport Titanium 1.5L TDCi,Ford,2014,"5,75,000","36,000 kms",Diesel


#Analysing the dataset

In [ ]:
print("Number of rows: ", car_data.shape[0])
print("Number of features: ", car_data.shape[1])

Number of rows:  892
Number of features:  6


In [ ]:
car_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 892 entries, 0 to 891
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        892 non-null    object
 1   company     892 non-null    object
 2   year        892 non-null    object
 3   Price       892 non-null    object
 4   kms_driven  840 non-null    object
 5   fuel_type   837 non-null    object
dtypes: object(6)
memory usage: 41.9+ KB


In [ ]:
car_data.describe()

,name,company,year,Price,kms_driven,fuel_type
count,892,892,892,892,840,837
unique,525,48,61,274,258,3
top,Honda City,Maruti,2015,Ask For Price,"45,000 kms",Petrol
freq,13,235,117,35,30,440


In [ ]:
car_data['fuel_type'].unique()

array(['Petrol', 'Diesel', nan, 'LPG'], dtype=object)

# Backup

In [ ]:
# Creating Backup
backup = car_data.copy()

#Preprocessing and Cleaning

## Issues I'm supposed to addess:

- names are pretty inconsistent
- names have company names attached to it
- some names are spam like 'Maruti Ertiga showroom condition with' and 'Well mentained Tata Sumo'
- company: many of the names are not of any company like 'Used', 'URJENT', and so on.
- year has many non-year values
- year is in object. Change to integer
- Price has Ask for Price
- Price has commas in its prices and is in object
- kms_driven has object values with kms at last.
- It has nan values and two rows have 'Petrol' in them
- fuel_type has nan values

In [ ]:
# Restoring the dataset
car_data = backup.copy()

In [ ]:
# Cleaning non numeric year values:
print(car_data['year'].dtype)
print(car_data['year'].unique())
car_data = car_data[car_data['year'].str.isnumeric()]
car_data['year'] = car_data['year'].astype(int)
print('Number of rows: ', car_data.shape[0])

object
['2007' '2006' '2018' '2014' '2015' '2012' '2013' '2016' '2010' '2017'
 '2008' '2011' '2019' '2009' '2005' '2000' '...' '150k' 'TOUR' '2003'
 'r 15' '2004' 'Zest' '/-Rs' 'sale' '1995' 'ara)' '2002' 'SELL' '2001'
 'tion' 'odel' '2 bs' 'arry' 'Eon' 'o...' 'ture' 'emi' 'car' 'able' 'no.'
 'd...' 'SALE' 'digo' 'sell' 'd Ex' 'n...' 'e...' 'D...' ', Ac' 'go .'
 'k...' 'o c4' 'zire' 'cent' 'Sumo' 'cab' 't xe' 'EV2' 'r...' 'zest']
Number of rows:  842


In [ ]:
car_data.head()

,name,company,year,Price,kms_driven,fuel_type
0,Hyundai Santro Xing XO eRLX Euro III,Hyundai,2007,"80,000","45,000 kms",Petrol
1,Mahindra Jeep CL550 MDI,Mahindra,2006,"4,25,000",40 kms,Diesel
2,Maruti Suzuki Alto 800 Vxi,Maruti,2018,Ask For Price,"22,000 kms",Petrol
3,Hyundai Grand i10 Magna 1.2 Kappa VTVT,Hyundai,2014,"3,25,000","28,000 kms",Petrol
4,Ford EcoSport Titanium 1.5L TDCi,Ford,2014,"5,75,000","36,000 kms",Diesel


In [ ]:
# Cleaning commas in Price and replacing 'Ask For Price' with NaN
car_data['Price'] = car_data['Price'].str.replace(',', '').replace('Ask For Price', float('nan'))

print('Number of rows: ', car_data.shape[0])
print(car_data.head())

# Convert the 'Price' column to numeric, coercing errors to NaN
car_data['Price'] = pandas.to_numeric(car_data['Price'], errors='coerce')

# Filter out rows where 'Price' is NaN and convert to integer
car_data = car_data.dropna(subset=['Price'])
car_data['Price'] = car_data['Price'].astype(int)

print('Number of rows after cleaning: ', car_data.shape[0])

Number of rows:  842
                                     name   company  year   Price  kms_driven  \
0    Hyundai Santro Xing XO eRLX Euro III   Hyundai  2007   80000  45,000 kms   
1                 Mahindra Jeep CL550 MDI  Mahindra  2006  425000      40 kms   
2              Maruti Suzuki Alto 800 Vxi    Maruti  2018     NaN  22,000 kms   
3  Hyundai Grand i10 Magna 1.2 Kappa VTVT   Hyundai  2014  325000  28,000 kms   
4        Ford EcoSport Titanium 1.5L TDCi      Ford  2014  575000  36,000 kms   

  fuel_type  
0    Petrol  
1    Diesel  
2    Petrol  
3    Petrol  
4    Diesel  
Number of rows after cleaning:  819


In [ ]:
# Cleaning kms_driven to remove commas, kms and two random petrol values then converting it to int
car_data['kms_driven'] = car_data['kms_driven'].str.split().str.get(0).str.replace(',', '')
car_data = car_data[car_data['kms_driven'].str.isnumeric()]
car_data['kms_driven'] = car_data['kms_driven'].astype(int)
print('Number of rows after cleaning: ', car_data.shape[0])

Number of rows after cleaning:  817


In [ ]:
# Cleaning nan fuel_type values
car_data.dropna(subset=['fuel_type'], inplace=True)
print('Number of rows after cleaning: ', car_data.shape[0])

Number of rows after cleaning:  816


In [ ]:
# Cleaning car names
car_data['name'] = car_data['name'].str.split().str.slice(start=0, stop=3).str.join(' ')
print(car_data.head())
print('Number of rows after cleaning: ', car_data.shape[0])

                     name   company  year   Price  kms_driven fuel_type
0     Hyundai Santro Xing   Hyundai  2007   80000       45000    Petrol
1     Mahindra Jeep CL550  Mahindra  2006  425000          40    Diesel
3       Hyundai Grand i10   Hyundai  2014  325000       28000    Petrol
4  Ford EcoSport Titanium      Ford  2014  575000       36000    Diesel
6               Ford Figo      Ford  2012  175000       41000    Diesel
Number of rows after cleaning:  816


In [ ]:
# Handling outliers
car_data = car_data[car_data['Price'] < 6e6]
print('Number of rows after cleaning: ', car_data.shape[0])

Number of rows after cleaning:  815


In [ ]:
# Resetting indexes
car_data = car_data.reset_index(drop=True)
car_data.head()

,name,company,year,Price,kms_driven,fuel_type
0,Hyundai Santro Xing,Hyundai,2007,80000,45000,Petrol
1,Mahindra Jeep CL550,Mahindra,2006,425000,40,Diesel
2,Hyundai Grand i10,Hyundai,2014,325000,28000,Petrol
3,Ford EcoSport Titanium,Ford,2014,575000,36000,Diesel
4,Ford Figo,Ford,2012,175000,41000,Diesel


In [ ]:
car_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 815 entries, 0 to 814
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        815 non-null    object
 1   company     815 non-null    object
 2   year        815 non-null    int64 
 3   Price       815 non-null    int64 
 4   kms_driven  815 non-null    int64 
 5   fuel_type   815 non-null    object
dtypes: int64(3), object(3)
memory usage: 38.3+ KB


In [ ]:
# Exporting as csv without index column
car_data.to_csv('car.csv')

# Building the model

In [ ]:
# Importing and Splitting training & testing data
new_data = pandas.read_csv('car.csv', index_col=0)
X = new_data.drop(columns='Price')
y = new_data['Price']
print('Shape of X: ', X.shape)
print('Shape of y: ', y.shape)

Shape of X:  (815, 5)
Shape of y:  (815,)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
# Encoding categorical columns
encoder = OneHotEncoder()
encoder.fit(X[['name', 'company', 'fuel_type']])

OneHotEncoder()

In [ ]:
column_transformer = make_column_transformer(
    (OneHotEncoder(categories=encoder.categories_), ['name', 'company', 'fuel_type']),
    remainder='passthrough'
)

In [ ]:
# Defining algorithm and pipeline
algorithm = LinearRegression()
pipeline = make_pipeline(column_transformer, algorithm)

In [ ]:
# Taining the entire pipeline on our training data
pipeline.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array(['Audi A3 Cabriolet', 'Audi A4 1.8', 'Audi A4 2.0', 'Audi A6 2.0',
       'Audi A8', 'Audi Q3 2.0', 'Audi Q5 2.0', 'Audi Q7', 'BMW 3 Series',
       'BMW 5 Series', 'BMW 7 Series', 'BMW X1', 'BMW X1 sDrive20d',
       'BMW X1 xDrive20d', 'Chevrolet Beat', 'Chevrolet Beat...
                                                                            array(['Audi', 'BMW', 'Chevrolet', 'Datsun', 'Fiat', 'Force', 'Ford',
       'Hindustan', 'Honda', 'Hyundai', 'Jaguar', 'Jeep', 'Land',
       'Mahindra', 'Maruti', 'Mercedes', 'Mini', 'Mitsubishi', 'Nissan',
       'Renault', 'Skoda', 'Tata', 'Toyota', 'Volkswagen', 'Volvo'],
      dtype=object),
                                                                            array(['Diesel', 'LPG', 'Petrol'], dtype=object)]),
                                                  ['name', 'company',
                                                   'fuel_type'])])),
                ('linearregression', LinearRegression())])

In [ ]:
  # Predicting values and analysing performance
y_pred = pipeline.predict(X_test)
r2_score(y_test, y_pred)

0.7567097232017798